Autor: Esteban Suárez Calvo

In [1]:
from datetime import timedelta
from time import time

from qiskit.circuit.library import PauliFeatureMap, z_feature_map, zz_feature_map
from qiskit_aer.primitives import SamplerV2 as Sampler
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

# CCAM - Práctica 3: Quantum Support Vector Machines

Antes de empezar, fijaremos la semilla aleatoria, de manera que los resultados obtenidos en el notebook sean reproducibles.

In [2]:
RANDOM_SEED = 13
algorithm_globals.random_seed = RANDOM_SEED

## Preprocesado

### Normalización del dataset

Normalizamos el dataset utilizando la normalización `MinMax`.

In [3]:
data = load_breast_cancer()
X = data.data
y = data.target
X = MinMaxScaler().fit_transform(X)
print(f"Número de características = {X.shape[1]}")

Número de características = 30


### Reducción de dimensionalidad

Dado que la simulación de circuitos cuánticos escala exponencialmente con el número de qubits, trabajar con 30 características iniciales resulta inviable.

Por lo tango, aplicamos la técnica ***Principal Component Analysis (PCA)*** para  reducir la dimensionalidad a **8 características**.

Aunque esto implica una pérdida de información, es un compromiso necesario para ajustar el problema procesable por el simulador en un tiempo razonable.

Como veremos en los siguientes apartados, dependiendo del optimizador escogido, el tiempo de entrenamiento puede ser bastante elevado (horas), mientras que en otros dura cuestión de minutos. Por lo tanto, aunque resulta posible utilizar un mayor número de características, hay ciertos optimizadores para los cuales no podríamos haber realizado la simulación.

Por lo tanto, al elegir 8 características, estamos buscando un equilibrio entre reducir la dimensionalidad para poder utilizar varios optimizadores diferentes, pero sin eliminar demasiada información.

In [4]:
X = PCA(n_components=8).fit_transform(X)
number_of_features = X.shape[1]
print(f"Nuevo número de características = {number_of_features}")

Nuevo número de características = 8


### Dividir el conjunto entre entrenamiento y test

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_SEED
)
print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de test: {len(X_test)}")

Datos de entrenamiento: 455
Datos de test: 114


## Desarrolla dos SVM

### Desarrolla un modelo clásico

Con el fin de probar diferentes modelos clásicos de forma cómoda, definimos las siguientes funciones para entrenar y evaluar un SVC clásico:

In [6]:
def train_SVC(kernel) -> SVC:
    svc = SVC(kernel=kernel)
    return _fit(svc)


def train_QSVC(feature_map) -> QSVC:
    sampler = Sampler(seed=RANDOM_SEED)
    fidelity = ComputeUncompute(sampler=sampler)
    kernel = FidelityQuantumKernel(fidelity=fidelity, feature_map=feature_map)
    qsvc = QSVC(quantum_kernel=kernel)
    return _fit(qsvc)


def _fit(svc: SVC) -> SVC:
    global X_train, Y_train

    start = time()
    svc.fit(X_train, Y_train)
    total_time = time() - start

    if total_time < 10:
        print(f"Tiempo total de entrenamiento: {total_time * 1000:.2} ms")
    else:
        print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return svc


def evaluate_SVC(svc: SVC) -> None:
    global X_train, Y_train, X_test, Y_test

    train_score = svc.score(X_train, Y_train)
    test_score = svc.score(X_test, Y_test)
    y_pred_svc = svc.predict(X_test)
    svc_recall = recall_score(Y_test, y_pred_svc, pos_label=0)

    print(f"SVC en entrenamiento: {train_score:.5}")
    print(f"SVC en test (Accuracy): {test_score:.5}")
    print(f"SVC en test (Recall): {svc_recall:.5}")

## Ejecutar los modelos con diferenes kernels/mapas de características

### SVM

In [7]:
svc = train_SVC("linear")
evaluate_SVC(svc)

Tiempo total de entrenamiento: 2.5 ms
SVC en entrenamiento: 0.97363
SVC en test (Accuracy): 0.99123
SVC en test (Recall): 0.97222


In [8]:
svc = train_SVC("poly")
evaluate_SVC(svc)

Tiempo total de entrenamiento: 3.3 ms
SVC en entrenamiento: 0.92088
SVC en test (Accuracy): 0.9386
SVC en test (Recall): 0.80556


In [9]:
svc = train_SVC("rbf")
evaluate_SVC(svc)

Tiempo total de entrenamiento: 2.2 ms
SVC en entrenamiento: 0.98022
SVC en test (Accuracy): 0.97368
SVC en test (Recall): 0.97222


### QSVM

In [ ]:
qsvc = train_QSVC(z_feature_map(number_of_features))
evaluate_SVC(qsvc)

Tiempo total de entrenamiento: 0:11:47.936480
SVC en entrenamiento: 0.96484
SVC en test (Accuracy): 0.96491
SVC en test (Recall): 0.94444


In [ ]:
qsvc = train_QSVC(zz_feature_map(number_of_features))
evaluate_SVC(qsvc)

Tiempo total de entrenamiento: 0:53:35.252097
SVC en entrenamiento: 1.0
SVC en test (Accuracy): 0.83333
SVC en test (Recall): 0.61111


In [ ]:
qsvc = train_QSVC(PauliFeatureMap(number_of_features).decompose())
evaluate_SVC(qsvc)

Tiempo total de entrenamiento: 0:42:16.785167
SVC en entrenamiento: 1.0
SVC en test (Accuracy): 0.78947
SVC en test (Recall): 0.61111


## Resultados

Agrupamos en las siguientes tablas los resultados obtenidos con los distintos clasificadores.

### SVC

|          | *Accuracy (train)* | *Accuracy (test)* | *Recall (test)* |
| -------- | ------------------ | ----------------- | --------------- |
| `linear` | 0.97363            | 0.99123           | 0.97222         |
| `poly`   | 0.92088            | 0.9386            | 0.80556         |
| `rbf`    | 0.98022            | 0.97368           | 0.97222         |

### QSVC

|                   | *Accuracy (train)* | *Accuracy (test)* | *Recall (test)* |
| ----------------- | ------------------ | ----------------- | --------------- |
| `z_feature_map`   | 0.96484            | 0.96491           | 0.94444         |
| `zz_feature_map`  | 1.0                | 0.83333           | 0.61111         |
| `PauliFeatureMap` | 1.0                | 0.78947           | 0.61111         |


## Conclusiones

### Análisis del caso Clásico y Contexto del Problema

El dataset utilizado (*Breast Cancer*) es un problema médico crítico donde la métrica más relevante es el **Recall** (sensibilidad). Dado que estamos diagnosticando cáncer, el coste de un "Falso Negativo" (no detectar un cáncer existente) es muy elevado.

-   **Linear & RBF:** Ambos modelos clásicos ofrecen un rendimiento excelente (~97-99% accuracy y 0.97 recall), lo que indica que el problema es linealmente separable o casi linealmente separable en el espacio de características reducido.
-   **Poly:** A pesar de tener una accuracy razonable (>90%), su recall cae al 80%. Esto lo invalida para este uso médico, demostrando que no basta con mirar la precisión global.

### Interpretación de los Resultados Cuánticos (QSVC)

-   **Z Feature Map (El mejor cuántico):** Este mapa de características no introduce entrelazamiento (*entanglement*), solo rotaciones locales. Curiosamente, es el único que generaliza bien (96% en test). Esto refuerza la idea de que la estructura de los datos es sencilla y no requiere correlaciones complejas para ser separada.
-   **ZZ & Pauli Feature Maps (Overfitting severo):** Estos mapas introducen entrelazamiento y crean un espacio de Hilbert mucho más complejo. Observamos un claro caso de **overfitting**:
    -   *Train Accuracy: 1.0 (Perfecto)* -> El modelo ha memorizado los datos de entrenamiento.
    -   *Test Accuracy: ~0.78-0.83 (Pobre)* -> El modelo falla al generalizar.
    -   **Causa:** La alta expresividad de estos mapas cuánticos, combinada con un dataset pequeño, hace que el modelo aprenda el "ruido" de los datos en lugar del patrón subyacente. En este caso, **más complejidad cuántica ha sido contraproducente**.

### Comparativa Final y Dimensionalidad

Hemos reducido el problema de 30 a 8 características mediante PCA. Dado que diversos modelos, tanto clásicos como cuánticos, obtienen un accuracy superior al 95% en test, concluimos que el problema sigue siendo resoluble a pesar de haber reducido tanto la dimensionalidad del problema.

Finalmente, **no hemos observado Ventaja Cuántica** en este experimento. El mejor modelo cuántico (`z_feature_map`) iguala el rendimiento de los modelos clásicos, pero con un coste computacional mayor. Esto es esperable en datasets clásicos que pueden ser resueltos eficientemente por métodos convencionales.